In [1]:
import numpy as np
import pandas as pd
import folium, json

import warnings
warnings.filterwarnings('ignore')

import requests
from bs4 import BeautifulSoup
from urllib.parse import quote
from tqdm.notebook import tqdm

import re
import time

import selenium
from selenium import webdriver
from selenium.webdriver.common.keys import Keys
from selenium.webdriver.common.by import By

In [3]:
from wordcloud import WordCloud, STOPWORDS # 주피터는 wordCloud 인것 같음
from PIL import Image # 그림을 불러오는 패키지
import numpy as np
import matplotlib.pyplot as plt
from nltk.corpus import stopwords
from konlpy.tag import Okt, Kkma, Komoran

from matplotlib import font_manager, rc 
import matplotlib.pyplot as plt
import matplotlib as mpl

#마이너스 표시
mpl.rcParams['axes.unicode_minus'] = False
plt.rc('font', family='Malgun Gothic')

In [7]:
# url
i = 1
keyword = "여친선물" 
url = f'https://kin.naver.com/search/list.naver?query={quote(keyword)}&page={i}'

req = requests.get(url)
soup = BeautifulSoup(req.text, 'html.parser')

In [93]:
answerList=[]
keyword = "여친선물" 

for i in range(1,11):
    url = f'https://kin.naver.com/search/list.naver?query={quote(keyword)}&page={i}'

    req = requests.get(url)
    soup = BeautifulSoup(req.text, 'html.parser')

    for li in soup.select(".basic1 li"):
        try: 
            driver = webdriver.Chrome()
            li_url = li.select_one('a')['href']
            driver.get(li_url)
            time.sleep(2)

            ans_num = int(driver.find_element(By.CSS_SELECTOR, ".c-classify__title em").text)
            
            if ans_num%5 == 0:
                plus = int(ans_num / 5)-1
            else :
                plus = int(ans_num / 5)

            for j in range(0,plus):
                driver.find_element(By.CSS_SELECTOR, "#nextPageButton").click()
                time.sleep(1)

            li_soup = BeautifulSoup(driver.page_source, 'html.parser')

            for p in li_soup.select("._endContents"):
                try : 
                    answerList.append(p.select_one(".se-module").text)
                except :
                    answerList.append(p.select_one("._endContentsText").text)
            
            driver.close()
            time.sleep(1)
        except :
            print(li_url)
            break

In [99]:
hw_answer = pd.DataFrame(answerList)
hw_answer.to_csv('hw_answer.csv', index=False)
hw_answer

,0
0,여친선물로​데일리로맨틱 오르골 무드등 강추요​여친선물 BEST입니다~^^ 후기봐보...
1,먼저 여친의 생일을 축하합니다!​20대 초반 여친 생일선물 추천드립니다.20대 여...
2,인기있는 여친선물 리스트 말고뭔가 특별한 선물이어야 하지 않을까요?​정성스런 손편...
3,"20대 초반 여자친구 선물로사랑 문답, 펜디 카드지갑, 꽃다발 추천드려요!!카드지..."
4,여자친구 선물 고민중이시면 목걸이 완전 추천해요!오랫동안 간직할수도 있고 여자친구...
...,...
621,꽃이고 반지고 목걸이고 남자들이 사는거 꺼려지는거내가 잘 알지. 글씨도 잘 못써서 ...
622,여친선물 추천사이트들 모아둔 곳 소개합니다!!여친선물에 대해서 알아보고 계시는군요^...
623,여자선물은 목걸이나 귀걸이가 최고아닌가요 ㅋㅋㅋㅋ 몇개추천함..다이아티니하트 목걸이...
624,선물 베스트 20 1. 공책2. 지갑(빨간게좋음)3. 커플링4. 색볼펜5. 스케치북...


## 강사님 방법

In [106]:
base_url = "https://kin.naver.com/search/list.naver?query="
url = base_url + quote("여친선물")
driver = webdriver.Chrome()

In [117]:
answer_list = []
driver = webdriver.Chrome()
for page in range(1,11):
    print('page', page)
    url = f'{base_url}{quote("여친선물")}&page={page}'
    driver.get(url)
    time.sleep(1)
    
    html = driver.page_source
    soup = BeautifulSoup(html, 'html.parser')
    lis = soup.select('ul.basic1>li')
    
    for li in lis:
        sub_url = li.select_one('a._searchListTitleAnchor')['href']
        driver.get(sub_url)
        time.sleep(1)
        
        count = int(driver.find_element(By.CSS_SELECTOR,"._answerCount.num").text)
        answer_page=(count-1)//5
        
        for i in range(answer_page):
            driver.find_element(By.ID, 'nextPageButton').click()
            time.sleep(1)
        
        html = driver.page_source
        soup = BeautifulSoup(html, 'html.parser')
        divs = soup.select('div.answer-content__item._contentWrap._answer')
        for div in divs:
            ps = div.select('p.se-text-paragraph')
            for p in ps:
                text = p.get_text()
                if text == " " or text == "\u200b":
                    continue
                answer_list.append(text)
        driver.back()
        time.sleep(1)

page 1
page 2
page 3
page 4
page 5
page 6
page 7
page 8
page 9
page 10


In [114]:
soup.select('ul.basic1>li')[0].select_one('a._searchListTitleAnchor')['href']

'https://kin.naver.com/qna/detail.naver?d1id=5&dirId=511&docId=443958205&qb=7Jes7Lmc7ISg66y8&enc=utf8&section=kin&rank=1&search_sort=0&spq=0'

In [119]:
with open('gf_gift.txt',"w",encoding='utf-8') as f :
    for answer in answer_list:
        f.write(answer + '\n')